## Proceso de ETL en el dataset australian_user_reviews 

#### Este proceso se realiza con la finalidad de leer el dataset con el formato correcto, eliminar las columnas que no se necesitan para responder las consultas y hacer las transformaciones necesarias para optimizar el rendimiento de la API y el entrenamiento del modelo de aprendizaje automático.

In [1]:
import pandas as pd
import json
import ast

#### El archivo JSON se carga en un DataFrame utilizando la función open() de Python para abrir el archivo y leer su contenido en una lista de cadenas de texto. Luego, utiliza una comprensión de lista para evaluar cada cadena de texto como una expresión de Python y convertirla en un diccionario.

In [2]:
data_list = []
file_path = "datasets_originales/australian_user_reviews.json"
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError:
            continue

### Se crea un DataFrame de pandas a partir de la lista de diccionarios.

In [3]:
df_reviews = pd.DataFrame(data_list)

### Visualización del DataFrame.

In [4]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


### Se desanida la columna reviews con el método .explode() para transformar cada elemento de una lista en una fila separada, replicando los valores de los índices.

In [6]:
df_reviews_act = df_reviews.explode('reviews')

### Se elimina la columna original “reviews” del DataFrame Luego, se aplica el método .apply(pd.Series) a la columna “reviews” para convertir cada elemento de la lista en columnas individuales. Finalmente, se concatena el DataFrame resultante con las nuevas columnas utilizan

In [7]:
df_reviews_act = pd.concat([df_reviews_act.drop(['reviews'], axis=1), df_reviews_act['reviews'].apply(pd.Series)], axis=1) 

### Mediante el método .extract() se crea la columna "posted_year" para buscar un patrón de expresión regular (regex) en cada celda de la columna “posted”. El patrón (\d{4}) busca exactamente cuatro dígitos consecutivos que representan un año.

In [8]:
df_reviews_act ['posted_year'] = df_reviews_act ['posted'].str.extract(r'(\d{4})')

### Se elimina la columna "posted".

In [9]:
df_reviews_act .drop('posted' , axis = 1, inplace = True)

### Visualización del DataFrame. 

In [10]:
df_reviews_act.head(2)

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,review,0,posted_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN,2011


### Con la libreria nlkt se utiliza VADER para analizar el sentimiento de las reseñas en la columna “review”, se asigna valores según la escala (positivo, negativo o neutro) y los puntajes son almacenados en la nueva columna “sentiment_analysis”.

In [11]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Inicializar el analizador de sentimientos
sia = SentimentIntensityAnalyzer()

# Función para asignar valores según la escala
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 2  # Positivo
        elif compound_score <= -0.05:
            return 0  # Negativo
        else:
            return 1
    else:
        return 1  # Return neutral for non-string values


# Convertir la columna 'review' a tipo de dato str
df_reviews_act['review'] = df_reviews_act['review'].astype(str)

# Aplicar la función get_sentiment_score a la columna 'review'
df_reviews_act['sentiment_analysis'] = df_reviews_act['review'].apply(get_sentiment_score)

# Imprimir el DataFrame actualizado
print(df_reviews_act)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\58424\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                 user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1                js41637               http://steamcommunity.com/id/js41637   
1                js41637               http://steamcommunity.com/id/js41637   
...                  ...                                                ...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                                  funny last_edited

### El DataFrame obtenido de los diferentes procesos se exporta a un archivo CSV.

In [12]:
df_reviews_act .to_csv('australian_user_reviews_clean.csv') 